In [1]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=2

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=2


In [2]:
import os
import pymatgen
import matplotlib.pyplot as plt
import pandas as pd
from tqdm.notebook import trange, tqdm
from data import get_dichalcogenides_innopolis_202105

In [3]:
structures = get_dichalcogenides_innopolis_202105()

  0%|          | 0/3480 [00:00<?, ?it/s]

/temporary/pymatgen/pymatgen/io/cif.py:1123: UserWarning:

Issues encountered while parsing CIF: Some fractional co-ordinates rounded to ideal values to avoid issues with finite precision.



In [4]:
import matplotlib

from megnet.models import MEGNetModel
from megnet.data.graph import GaussianDistance
from megnet.data.crystal import CrystalGraph
from megnet.data.molecule import MolecularGraph
from megnet.utils.preprocessing import StandardScaler
from megnet.callbacks import ModelCheckpointMAE
from pymatgen.core import Lattice, Structure, Molecule

import tensorflow as tf
import numpy as np

In [5]:
import wandb
from wandb.keras import WandbCallback

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
train, test = train_test_split(structures, test_size=0.25, random_state=42)

In [8]:
wandb.init(project='ai4material_design', entity='kazeev')

wandb: Currently logged in as: kazeev (use `wandb login --relogin` to force relogin)


In [9]:
config = wandb.config
config.target = "homo"

In [10]:
gc = CrystalGraph(bond_converter=GaussianDistance(np.linspace(0, 5, 100), 0.5), cutoff=6)
model = MEGNetModel(nfeat_edge=100, nfeat_global=2, graph_converter=gc, npass=5)

In [11]:
scaler = StandardScaler.from_training_data(train["initial_structure"],
                                           train[config.target], is_intensive=True)
model.target_scaler = scaler

In [ ]:
model.train(train["initial_structure"], train[config.target],
            validation_structures=test["initial_structure"],
            validation_targets=test[config.target],
            callbacks=[WandbCallback()],
            epochs=1000, verbose=1, patience=1000, prev_model="callback/val_mae_00079_0.069863.hdf5")

Epoch 1/1000


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:437: UserWarning:

Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model/set2set_atom/Reshape_9:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/model/set2set_atom/Reshape_8:0", shape=(None,), dtype=float32), dense_shape=Tensor("gradient_tape/model/set2set_atom/Cast:0", shape=(1,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:437: UserWarning:

Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model/set2set_bond/Reshape_9:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/model/set2set_bond/Reshape_8:0", shape=(None,), dtype=float32), dense_shape=Tensor("gradient_tape/model/set2set_bond/Cast:0", shape=(1,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of m

21/21 [==============================] - 82s 3s/step - loss: 0.0972


INFO:megnet.callbacks:
Epoch 00001: val_mae improved from inf to 0.07860, saving model to callback/val_mae_00001_0.078596.hdf5


Epoch 2/1000
21/21 [==============================] - 59s 3s/step - loss: 0.0852


...

Epoch 13/1000
21/21 [==============================] - 89s 4s/step - loss: 0.0787
Epoch 14/1000
21/21 [==============================] - 85s 4s/step - loss: 0.0786
Epoch 15/1000
21/21 [==============================] - 88s 4s/step - loss: 0.0769
Epoch 16/1000
21/21 [==============================] - 92s 4s/step - loss: 0.0765
Epoch 17/1000
21/21 [==============================] - 88s 4s/step - loss: 0.0770
Epoch 18/1000
21/21 [==============================] - 89s 4s/step - loss: 0.0773
Epoch 19/1000
21/21 [==============================] - 87s 4s/step - loss: 0.0791
Epoch 20/1000
21/21 [==============================] - 90s 4s/step - loss: 0.0822
Epoch 21/1000
21/21 [==============================] - 88s 4s/step - loss: 0.0791


INFO:megnet.callbacks:
Epoch 00021: val_mae improved from 0.06969 to 0.06930, saving model to callback/val_mae_00021_0.069304.hdf5


Epoch 22/1000
21/21 [==============================] - 87s 4s/step - loss: 0.0792


INFO:megnet.callbacks:
Epoch 00022: val_mae improved from 0.06930 to 0.06909, saving model to callback/val_mae_00022_0.069086.hdf5


Epoch 23/1000
21/21 [==============================] - 93s 4s/step - loss: 0.0736
Epoch 24/1000
21/21 [==============================] - 91s 4s/step - loss: 0.0749
Epoch 25/1000
16/21 [=====================>........] - ETA: 21s - loss: 0.0762